# Ch10. 워드 임베딩 (Word Embedding)

# v05. 사전 훈련된 워드 임베딩 (Pre-trained Word Embedding)

- 이번 챕터에서는 **케라스의 임베딩 층(embedding layer)**과 **사전 훈련된 워드 임베딩(pre-trained word embedding)**을 가져와서 사용하는 것을 비교한다.
- 자연어 처리를 구현하려고 할 때 갖고 있는 훈련 데이터의 단어들을 임베딩 층(embedding layer)을 구현하여 임베딩 벡터로 학습하는 경우가 있다.
- 케라스에서는 이를 `Embedding()`이라는 도구를 사용하여 구현한다.

- 그런데 위키피디아 등과 같은 방대한 코퍼스를 가지고 Word2Vec, FastText, GloVe 등을 통해서 이미 미리 훈련된 임베딩 벡터를 불러오는 방법을 사용하는 경우도 있다.
- 이는 현재 갖고 있는 훈련 데이터를 임베딩 층으로 처음부터 학습을 하는 방법과는 대조된다.

<br>

## 5.1 케라스 임베딩 층 (Keras Embedding layer)

- 케라스는 훈련 데이터의 단어들에 대해 워드 임베딩을 수행하는 도구 `Embedding()`을 제공한다.
- `Embedding()`은 인공 신경망 구조 관점에서 임베딩 층(embedding layer)을 구현한다.

<br>

### 5.1.1 임베딩 층은 룩업 테이블이다.

- 임베딩 층의 입력으로 사용하기 위해서 입력 시퀀스의 각 단어들은 모두 정수 인코딩이 되어야 한다.

$
\qquad
\text{어떤 단어} \; \rightarrow \; 
\text{단어에 부여된 고유한 정수값} \; \rightarrow \; 
\text{임베딩 층 통과} \; \rightarrow \; 
\text{밀집 벡터}
$

- 임베딩 층은 입력 정수에 대해 밀집 벡터(dense vector)로 맵핑한다.
- 이 밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련된다.
- 훈련 과정에서 단어는 모델이 풀고자하는 작업에 맞는 값으로 업데이트된다.
- 그리고 이 밀집 벡터를 임베딩 벡터라고 부른다.

- 정수를 밀집 벡터 또는 임베딩 벡터로 맵핑한다는 것은 어떤 의미일까?
- 특정 단어와 맵핑되는 정수를 인덱스로 가지는 테이블로부터 임베딩 벡터 값을 가져오는 룩업 테이블이라고 볼 수 있다.
- 그리고 이 테이블은 단어 집합의 크기만큼의 행을 가지므로 모든 단어는 고유한 임베딩 벡터를 가진다.

$\qquad$ ![](https://wikidocs.net/images/page/33793/lookup_table.PNG)

- 위의 그림은 단어 "great"이 정수 인코딩된 후 테이블로부터 해당 인덱스에 위치한 임베딩 벡터를 꺼내오는 모습을 보여준다.
- 위의 그림에서는 임베딩 벡터의 차원이 4로 설정되어 있다.
- 그리고 단어 "great"은 정수 인코딩 과정에서 1,918의 정수로 인코딩되었다.
- 그에 따라 단어 집합의 크기 만큼의 행을 가지는 테이블에서 인덱스 1,918번에 위치한 행을 단어 "great"의 임베딩 벡터로 사용한다.
- 이 임베딩 벡터는 모델의 입력이 되고, 역전파 과정에서 단어 "great"의 임베딩 벡터값이 학습된다.

- 룩업 테이블의 개념을 이론적으로 우선 접하고, 처음 케라스를 배울 때 어떤 분들은 임베딩 층의 원-핫 벡터가 아니어도 동작한다는 점에 헷갈려 한다.
- 케라스는 단어를 정수 인덱스로 바꾸고, 원-핫 벡터로 한 번 더 바꾸고 나서 임베딩 층의 입력으로 사용하는 것이 아니다.
- 단어를 정수 인덱스로만 바꾼 채로 임베딩 층의 입력으로 사용해도 룩업 테이블된 결과인 임베딩 벡터를 리턴한다.

- 케라스의 임베딩 층 구현 코드는 다음과 같다.

```python
# 아래의 각 인자는 저자가 임의로 선정한 숫자들이며 의미 있는 선정 기준이 아님
v = Embedding(20000, 128, input_length=500)
# vocab_size = 20000
# output_dim = 128
# input_length = 500
```

- 임베딩 층은 다음과 같은 세 개의 인자를 받는다.
  1. `vocab_size` : 텍스트 데이터의 전체 단어 집합의 크기이다.
  2. `output_dim` : 워드 임베딩 후의 임베딩 벡터의 차원이다.
  3. `input_length` : 입력 시퀀스의 길이이다. 만약 갖고 있는 각 샘플의 길이가 500개의 단어로 구성되어 있다면 이 값은 500이 된다.

- `Embedding()`은 `(number of samples, input_length)`인 2D 정수 텐서를 입력받는다.
- 이 때 각 sample은 정수 인코딩된 결과로, 정수의 시퀀스이다.

- `Embedding()`은 워드 임베딩 작업을 수행하고 `(number of samples, input_length, embeddingi word dimentionality)`인 3D 실수 텐서를 리턴한다.

<br>

### 5.1.2 임베딩 층 사용하기

- RNN 챕터에서 이미 사용한 바 있지만, 임베딩 층을 복습해보자.
- 문장의 긍, 부정을 판단하는 감성 분류 모델을 만들어 보자.

<br>

#### 5.1.2.1 필요 라이브러리 임포트

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

<br>

#### 5.1.2.2 데이터 생성

- 문장과 레이블 데이터를 생성한다.
- 긍정인 문장은 레이블 1, 부정인 문장은 레이블이 0이다.

In [ ]:
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

<br>

#### 5.1.2.3 토큰화 수행

- 케라스의 `Tokenizer()`를 사용하여 토큰화를 진행한다.

In [13]:
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1

print(vocab_size)

16


<br>

#### 5.1.2.4 정수 인코딩 수행

- 각 문장에 대해서 정수 인코딩을 수행한다.

In [14]:
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


<br>

#### 5.1.2.5 패딩

- 문장 중에서 가장 길이가 긴 문장의 길이는 4이다.

In [15]:
max_len = max(len(l) for l in X_encoded)
print(max_len)

4


<br>

- 모든 문장을 패딩하여 길이를 4로 만들어 준다.

In [16]:
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)

print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


<br>

#### 5.1.2.6 모델 설계

- 출력층에 1개의 뉴런에 활성화 함수로는 시그모이드 함수를 사용하여 이진 분류를 수행한다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len)) # 모든 임베딩 벡터는 4차원이다.
model.add(Flatten()) # Dense의 입력으로 넣기 위함
model.add(Dense(1, activation='sigmoid'))

<br>

#### 5.1.2.7 모델 훈련 및 평가

- 테스트 데이터에 대한 정확도가 아니며 훈련 데이터도 양이 적어서 정확도에 의미는 없다.
- 하지만 여기서 말하고자 하는 점은 현재 각 단어들의 임베딩 벡터들의 값은 학습 과정에서 다른 가중치들과 함께 학습된 값이다.

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

```
Epoch 1/100
1/1 - 0s - loss: 0.7002 - acc: 0.4286
Epoch 2/100
1/1 - 0s - loss: 0.6987 - acc: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6971 - acc: 0.5714

...중략...

Epoch 98/100
1/1 - 0s - loss: 0.5297 - acc: 1.0000
Epoch 99/100
1/1 - 0s - loss: 0.5276 - acc: 1.0000
Epoch 100/100
1/1 - 0s - loss: 0.5255 - acc: 1.0000
<tensorflow.python.keras.callbacks.History at 0x7f0db02e5c18>
```

<br>

## 5.2 사전 훈련된 워드 임베딩(Pre-Trained Word Embedding) 사용하기

- 임베딩 벡터를 얻기 위해서 케라스의 `Embedding()`을 사용하기도 하지만, 때로는 이미 훈련되어져 있는 워드 임베딩을 불러서 이를 임베딩 벡터로 사용하기도 한다.
- 훈련 데이터가 적은 상황이라면 모델에 케라스의 `Embedding()`을 사용하는 것보다 다른 텍스트 데이터로 사전 훈련되어 있는 임베딩 벡터를 불러오는 것이 나은 선택일 수 있다.

- 훈련 데이터가 적다면 케라스의 `Embedding()`으로 해당 문제에 충분히 특화된 임베딩 벡터를 만들어내는 것이 쉽지 않다.
- 차라리 해당 문제에 특화된 임베딩 벡터를 만드는 것이 어렵다면, 해당 문제에 특화된 것은 아니지만 보다 일반적이고 보다 많은 훈련 데이터로 이미 Word2Vec이나 GloVe 등으로 학습되어져 있는 임베딩 벡터들을 사용하는 것이 성능의 개선을 가져올 수 있다.

- 사전 훈련된 GloVe와 Word2Vec 임베딩을 사용해서 모델을 훈련시키는 실습을 진행해보자.
- [GloVe 다운로드 링크](http://nlp.stanford.edu/data/glove.6B.zip)
- [Word2Vec 다운로드 링크](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM)

- 훈련 데이터는 앞서 사용했던 데이터에 동일한 전처리가 수행된 상태라고 가정한다.

In [19]:
print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [20]:
print(y_train)

[1 0 0 1 1 0 1]


<br>

### 5.2.1 사전 훈련된 GloVe 사용하기

- 이제 임베딩 층을 설계하기 위한 과정부터 달라진다.
- 우선 다운로드 받은 파일인 `glove.6B.zip`의 압축을 풀면 그 안에 4개의 파일이 있다.
- 여기서 사용할 파일은 `glove.6B.100d.txt` 파일이다.
- 해당 파일은 하나의 줄 당 101개의 값을 가지는 리스트를 갖고 있다.
- 두 개의 줄만 읽어보자.

In [22]:
n = 0
f = open('glove.6B.100d.txt', encoding='utf8')

for line in f:
    word_vector = line.split() # 각 줄을 읽어와서 word_vector에 저장
    print(word_vector) # 각 줄을 출력
    word = word_vector[0] # word_vector에서 첫 번째 값만 저장
    print(word) # word_vector의 첫 번째 값만 출력
    n = n+1
    if n==2:
        break

f.close()

['the', '-0.038194', '-0.24487', '0.72812', '-0.39961', '0.083172', '0.043953', '-0.39141', '0.3344', '-0.57545', '0.087459', '0.28787', '-0.06731', '0.30906', '-0.26384', '-0.13231', '-0.20757', '0.33395', '-0.33848', '-0.31743', '-0.48336', '0.1464', '-0.37304', '0.34577', '0.052041', '0.44946', '-0.46971', '0.02628', '-0.54155', '-0.15518', '-0.14107', '-0.039722', '0.28277', '0.14393', '0.23464', '-0.31021', '0.086173', '0.20397', '0.52624', '0.17164', '-0.082378', '-0.71787', '-0.41531', '0.20335', '-0.12763', '0.41367', '0.55187', '0.57908', '-0.33477', '-0.36559', '-0.54857', '-0.062892', '0.26584', '0.30205', '0.99775', '-0.80481', '-3.0243', '0.01254', '-0.36942', '2.2167', '0.72201', '-0.24978', '0.92136', '0.034514', '0.46745', '1.1079', '-0.19358', '-0.074575', '0.23353', '-0.052062', '-0.22044', '0.057162', '-0.15806', '-0.30798', '-0.41625', '0.37972', '0.15006', '-0.53212', '-0.2055', '-1.2526', '0.071624', '0.70565', '0.49744', '-0.42063', '0.26148', '-1.538', '-0.30223

In [23]:
print(type(word_vector))
print(len(word_vector))

<class 'list'>
101


- 101개의 값 중에서 첫 번째 값은 임베딩 벡터가 의미하는 단어를 의미한다.
- 두 번째부터 마지막 값은 해당 단어의 임베딩 벡터의 100개의 차원에서의 각 값을 의미한다.
- 즉, `glove.6B.100d.txt`는 수많은 단어에 대해서 100개의 차원을 가지는 임베딩 벡터로 제공하고 있다.
- 위의 출력 결과는 단어 'the'에 대해서 100개의 차원을 가지는 임베딩 벡터와 단어 ','에 대해서 100개의 차원을 가지는 임베딩 벡터를 보여준다.

<br>

- 그러면 이제 `glove.6B.100d.txt`에 있는 모든 임베딩 벡터들을 불러와보자.
- 형식은 키(key)와 값(value)의 쌍(pair)를 가지는 파이썬의 사전형 구조를 사용한다.

In [24]:
import numpy as np

embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding='utf8')

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr

f.close()

print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


<br>

- 임의의 단어 'respectable'에 대해서 임베딩 벡터를 출력해보자.

In [25]:
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

- 벡터값이 출력되며 길이는 100인 것을 확인할 수 있다.

<br>

- 이제 훈련 데이터의 단어 집합의 모든 단어에 대해서 사전 훈련된 GloVe의 임베딩 벡터들을 맵핑한다.

In [26]:
# 단어 집합의 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
embedding_matrix = np.zeros((vocab_size, 100))
np.shape(embedding_matrix)

(16, 100)

In [27]:
print(t.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [ ]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = embedding_dict.get(word) # 단어(key)에 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp # 임시 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입

<br>

- 이제 이를 이용하여 임베딩 층(embedding layer)를 만들어보자.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)

- 현재 실습에서 사전 훈련된 워드 임베딩을 100차원의 값인 것으로 사용하고 있기 때문에 임베딩 층의 `output_dim`의 인자값으로 100을 주어야 한다.

- 그리고 사전 훈련된 워드 임베딩을 그대로 사용할 것이므로, 별도로 더 이상 훈련을 하지 않는다는 옵션을 준다.
- 이는 `trainable=False`로 선택할 수 있다.

In [ ]:
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

```
Epoch 1/100
1/1 - 0s - loss: 0.7460 - acc: 0.2857
Epoch 2/100
1/1 - 0s - loss: 0.7270 - acc: 0.2857
Epoch 3/100
1/1 - 0s - loss: 0.7086 - acc: 0.2857

...

Epoch 98/100
1/1 - 0s - loss: 0.1155 - acc: 1.0000
Epoch 99/100
1/1 - 0s - loss: 0.1140 - acc: 1.0000
Epoch 100/100
1/1 - 0s - loss: 0.1126 - acc: 1.0000
<tensorflow.python.keras.callbacks.History at 0x7f244d3e75c0>
```

<br>

- 사전 훈련된 GloVe 임베딩에 대한 예제는 아래의 케라스 블로그 링크에도 기재되어 있다.
  - [링크](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)

<br>

### 5.2.2 사전 훈련된 Word2Vec 사용하기

In [ ]:
import numpy as np
import gensim

In [33]:
# 현재 위치에 구글의 사전 훈련된 Word2Vec을 다운로드
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-03-31 03:48:59--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.40.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.40.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  63.6MB/s    in 24s     

2020-03-31 03:49:23 (66.0 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [34]:
# 구글의 사전 훈련된 Word2Vec 모델을 로드한다.
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',
                                                                 binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [35]:
print(word2vec_model.vectors.shape) # 모델의 크기 확인 

(3000000, 300)


- 300의 차원을 가진 Word2Vec 벡터가 3,000,000개 있다.

<br>

- 모든 값이 0으로 채워진 임베딩 행렬을 만들어준다.
- 이번 문제의 단어는 총 16개이므로, 16 x 300의 크기를 가진 행렬을 만든다.

In [36]:
embedding_matrix = np.zeros((vocab_size, 300))
np.shape(embedding_matrix)

(16, 300)

<br>

- `word2vec_model`에서 특정 단어를 입력하면 해당 단어의 임베딩 벡터를 리턴받을 것이다.
- 만약 `word2vec_model`에 특정 단어의 임베딩 벡터가 없다면 `None`을 리턴하도록 한다.

In [ ]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

<br>

- 단어 집합으로부터 단어를 1개씩 호출하여 `word2vec_model`에 해당 단어의 임베딩 벡터값이 존재하는 지 확인한다.
- 만약 `None`이 아니라면 존재한다는 의미이므로 임베딩 행렬에 해당 단어의 인덱스 위치의 행에 임베딩 벡터의 값을 저장한다.

In [ ]:
for word, i in t.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

- 이렇게 되면 현재 풀고자하는 문제의 16개의 단어와 맵핑되는 임베딩 행렬이 완성된다.

<br>

- 제대로 맵핑이 됐는 지 확인해보자.
- 기존 `word2vec_model`에 저장되어 있던 단어 'nice'의 임베딩 벡터값을 확인해보자.

In [ ]:
print(word2vec_model['nice'])

```
[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
    ....
 -0.16894531 -0.08642578 -0.08544922  0.18945312 -0.14648438  0.13476562
 -0.04077148  0.03271484  0.08935547 -0.26757812  0.00836182 -0.21386719]
```

- 이 단어 'nice'는 현재 단어 집합에서 몇 번 인덱스를 가지는 지 확인해보자.

In [40]:
print('단어 nice의 정수 인덱스 : ', t.word_index['nice'])

단어 nice의 정수 인덱스 :  1


- 1의 값을 가지므로 `embedding_matrix`의 1번 인덱스에는 단어 'nice'의 임베딩 벡터값이 있어야 한다.

In [ ]:
print(embedding_matrix[1])

```
[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
        ...
 -0.16894531 -0.08642578 -0.08544922  0.18945312 -0.14648438  0.13476562
 -0.04077148  0.03271484  0.08935547 -0.26757812  0.00836182 -0.21386719]
 ```

- 값이 `word2vec_model`에서 확인했던 것과 동일한 것을 확인할 수 있다.

<br>

- 이제 `Embedding`에 사전 훈련된 `embedding_matrix`를 입력으로 넣어주고 모델을 학습시켜 보자.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

```
Epoch 1/100
1/1 - 0s - loss: 0.6413 - acc: 0.7143
Epoch 2/100
1/1 - 0s - loss: 0.6246 - acc: 0.7143
Epoch 3/100
1/1 - 0s - loss: 0.6084 - acc: 0.8571
...
Epoch 98/100
1/1 - 0s - loss: 0.1075 - acc: 1.0000
Epoch 99/100
1/1 - 0s - loss: 0.1062 - acc: 1.0000
Epoch 100/100
1/1 - 0s - loss: 0.1049 - acc: 1.0000
<tensorflow.python.keras.callbacks.History at 0x7f241538ab38>
```